#### 1-INSTALANDO PACOTES

In [1]:
# Importando as bibliotecas
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from google.cloud import storage

# Comando para ignorar avisos em geral
import warnings
warnings.filterwarnings('ignore')

# Configuração da quantidade de colunas que vão aparecer em uma dataframe
pd.set_option('display.max_columns',100)

ETL - (Extração) - Dados Brutos 

In [2]:
# df = dataframe ou tabela
arquivo = (r"C:\Cursos e documentos\Bootcamp\Projeto - 01\data\raw\airbnbbruto.csv").replace('\\', '/' )
df = pd.read_csv(arquivo)

# visuaizando oo dataframe
display(df)

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,0,194.033698,Private room,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,1,344.245776,Private room,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,2,264.101422,Private room,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,3,433.529398,Private room,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,4,485.552926,Private room,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,1098,2486.115342,Entire home/apt,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,181.198355,9.594573,225.304203,15.699521,4.85869,52.37677
1099,1099,233.637194,Private room,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,108.024880,5.719989,134.775277,9.391335,4.83611,52.34910
1100,1100,317.062311,Private room,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,440.476515,23.323524,625.947562,43.616927,4.88897,52.37798
1101,1101,1812.855904,Entire home/apt,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,257.948105,13.658524,336.589237,23.454022,4.90688,52.35794


### 2 - Transformação 

#### Backup - 01

In [3]:
# Cópia de segurança caso ocorra algum erro
df_backup01 = df.copy()

* TRADUÇÃO-(RENOMENANDO ATRIBUTOS)

In [7]:
# Traduzindo os atributos para um melhor entendimento dos dados 
df.rename(columns={'realSum':'preço_total', 'room_type':'tipo_quarto', 'room_shared':'quarto_compartilhado', 'room_private':'quarto_privado', 
    'person_capacity':'capacidade', 'host_is_superhost':'superhost', 'cleanliness_rating':'nota_limpeza', 'guest_satisfaction_overall':'satisfaçao', 
    'bedrooms':'n_quartos', 'dist':'distancia_centro', 'metro_dist':'distancia_metro', 'attr_index':'atraçao_local_anuncio','attr_index_norm':'atraçao',
    'rest_index':'restaurantes_localização', 'rest_index_norm':'atração_restaurantes', 'lng':'longitude', 'lat':'latitude'}, inplace = True)

# Exibindo o dataframe traduzido - comando head('argumento quantidade de registros')
df.head(10)  

,Unnamed: 0,preço_total,tipo_quarto,quarto_compartilhado,quarto_privado,capacidade,superhost,multi,biz,nota_limpeza,satisfaçao,n_quartos,distancia_centro,distancia_metro,atraçao_local_anuncio,atraçao,restaurantes_localização,atração_restaurantes,longitude,latitude
0,0,194.033698,Private room,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,1,344.245776,Private room,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,2,264.101422,Private room,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,3,433.529398,Private room,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,4,485.552926,Private room,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508
5,5,552.808567,Private room,False,True,3.0,False,0,0,8.0,100.0,2,2.131420,1.904668,174.788957,9.255191,225.201662,15.692376,4.87699,52.38966
6,6,215.124317,Private room,False,True,2.0,False,0,0,10.0,94.0,1,1.881092,0.729747,200.167652,10.599010,242.765524,16.916251,4.91570,52.38296
7,7,2771.307384,Entire home/apt,False,False,4.0,True,0,0,10.0,100.0,3,1.686807,1.458404,208.808109,11.056528,272.313823,18.975219,4.88467,52.38749
8,8,1001.804420,Entire home/apt,False,False,4.0,False,0,0,9.0,96.0,2,3.719141,1.196112,106.226456,5.624761,133.876202,9.328686,4.86459,52.40175
9,9,276.521454,Private room,False,True,2.0,False,1,0,10.0,88.0,1,3.142361,0.924404,206.252862,10.921226,238.291258,16.604478,4.87600,52.34700


In [8]:
# Visualizando os unicos tipos de registros do atributo tipo_quarto
df['tipo_quarto'].unique()

array(['Private room', 'Entire home/apt', 'Shared room'], dtype=object)

In [10]:
# Criando um dicionário com a tradução dos tipos de quartos existentes dos registros do atributo tipo_quarto
tradução_tipo_quarto = {'Private room':'quarto privado', 'Entire home/apt':'Casa/apto inteiro','Shared room':'quarto compartilhado'} 

# Utilizando o dataframe(df) para localizar a coluna(atributo) tipo_quarto e fazendo a tradução utilizando o dicionário
df['tipo_quarto'].replace(tradução_tipo_quarto, inplace = True)

# Confirmando se está tudo certo utilizando a exibição
df['tipo_quarto'].unique()   

array(['quarto privado', 'Casa/apto inteiro', 'quarto compartilhado'],
      dtype=object)

In [18]:
# visualizando as últimas linhas do df - dataframe
df.tail()

,Unnamed: 0,preço_total,tipo_quarto,quarto_compartilhado,quarto_privado,capacidade,superhost,multi,biz,nota_limpeza,satisfaçao,n_quartos,distancia_centro,distancia_metro,atraçao_local_anuncio,atraçao,restaurantes_localização,atração_restaurantes,longitude,latitude
1098,1098,2486.115342,Casa/apto inteiro,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,181.198355,9.594573,225.304203,15.699521,4.85869,52.37677
1099,1099,233.637194,quarto privado,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,108.024880,5.719989,134.775277,9.391335,4.83611,52.34910
1100,1100,317.062311,quarto privado,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,440.476515,23.323524,625.947562,43.616927,4.88897,52.37798
1101,1101,1812.855904,Casa/apto inteiro,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,257.948105,13.658524,336.589237,23.454022,4.90688,52.35794
1102,1102,258.008577,quarto compartilhado,True,False,2.0,False,0,0,8.0,90.0,1,0.280204,0.516217,533.962653,28.273677,807.492307,56.267226,4.89295,52.37575


In [21]:
# Visualizando o df de forma aleatória para achar possíveis inconsistências
# sample argumento número de registros que irão aparecer a executar aleatoriamente
df.sample(5)

,Unnamed: 0,preço_total,tipo_quarto,quarto_compartilhado,quarto_privado,capacidade,superhost,multi,biz,nota_limpeza,satisfaçao,n_quartos,distancia_centro,distancia_metro,atraçao_local_anuncio,atraçao,restaurantes_localização,atração_restaurantes,longitude,latitude
460,460,1804.185316,Casa/apto inteiro,False,False,6.0,False,1,0,9.0,94.0,3,0.433109,0.227969,765.245811,40.520274,793.567194,55.296904,4.89932,52.37405
454,454,1167.248612,Casa/apto inteiro,False,False,4.0,True,0,0,10.0,98.0,2,1.976076,0.354117,488.221922,25.851675,455.014485,31.706064,4.88691,52.35588
655,655,958.451480,Casa/apto inteiro,False,False,6.0,False,0,1,9.0,96.0,3,7.150917,3.946522,60.765465,3.217572,77.635768,5.409772,4.99559,52.35926
593,593,618.189487,Casa/apto inteiro,False,False,4.0,False,0,0,8.0,90.0,2,2.144272,1.803146,213.094967,11.283520,267.870131,18.665576,4.86172,52.37163
384,384,344.245776,quarto privado,False,True,2.0,True,0,0,10.0,98.0,1,0.280446,0.273396,674.746584,35.728280,794.482351,55.360673,4.89682,52.37215
